In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import math
import scipy.stats
from sklearn import preprocessing
import pickle

#=============================read data=====================================

fname = "data_preprocessed_dic.p"
infile = open(fname, 'rb')
new_dict = pickle.load(infile)
infile.close()



Y = new_dict['Y_tmp']



XA = new_dict['XA_tmp']
XB = new_dict['XB_tmp']
# # drop linearly dependent columns
# q1,r1 = np.linalg.qr(XA_tmp.T)
# XA = XA_tmp[:, np.abs(np.diag(r1))>=1e-10]

# q2,r2 = np.linalg.qr(XB_tmp.T)
# XB = XB_tmp[:, np.abs(np.diag(r2))>=1e-10]

# add intercept terms that consist of 1 to the predictor datasets
# XA matrix with intercept
XAintercept = sm.add_constant(XA)

Model = 'logistic'
# output AUC
reportAUC = True
# do not report ll 
reportll = True
#========================Define the log-likelihood for evaluation========================
# logistic regression log-likelihood
def loglikeli(YEval, predCombined):
    # # clip the predicted probability to avoid it getting too close to 0 or 1
    # predCombined_clipped = np.clip(predCombined, 1e-15, (1-1e-15))

    predCombined_clipped = predCombined
    ll = np.mean(YEval * np.log(predCombined_clipped ) + (1-YEval) * np.log(1 - predCombined_clipped ))
    return ll
#=============================function to calculate the inverse logit link============================
def invLink(lp):
    pv = 1/(1 + math.exp(-lp))
    return pv

# vectorize the function
invLinkVec = np.vectorize(invLink)

#======================function to generate response data given linear predictors=====================
def ResponseGen(lp):
    pv = invLink(lp)
    binoRand = np.random.binomial(1, pv, 1)
    return binoRand

# vectorize the function
ResponseGenVec = np.vectorize(ResponseGen)

#=============================specify the model to fit============================
fitmodel = sm.families.Binomial(link = sm.families.links.logit())

In [2]:

def TestFun(testtype, Lapscale, K, Y, XAintercept, XB):
    # sample size
    n = XAintercept.shape[0]
    if Model == "logcosh":
        #========================define functions for fitting log-cosh====================
        logcoshA = 0.3
        def gradCal(Y, XAintercept, beta0):
            # get the residual
            resid = Y - np.dot(XAintercept, beta0)
            # calculate the gradient
            grad = np.mean((np.tanh(logcoshA * resid).reshape(n,-1)) * XAintercept, axis = 0)
            return(grad)

        def HessCal(Y, XAintercept, beta0):
            # get the residual
            resid = Y - np.dot(XAintercept, beta0)
            # calculate the Hessian
            Hess = logcoshA * n**(-1) * np.dot(np.dot(np.transpose(XAintercept), np.diag(np.cosh(logcoshA * resid)**(-1))), XAintercept)
            return(Hess)
        def fitLogCosh(Y, XAintercept):
            # fit a linear regression as the initial value
            model = sm.GLM(endog = Y, exog = XAintercept, family = fitmodel)
            # get initial value
            beta0 = model.fit().params

            # evaluate the gradient
            grad_updated = gradCal(Y, XAintercept, beta0)
            gradL2 = np.mean(grad_updated**2)

            # set convergence threthold
            thre = 1e-15
            # set counter
            ct = 0
            while (gradL2 > thre) | (ct <100):
                grad = gradCal(Y, XAintercept, beta0)
                Hess = HessCal(Y, XAintercept, beta0)
                beta0 = beta0 + np.dot(np.linalg.inv(Hess), grad)
                grad_updated = gradCal(Y, XAintercept, beta0)
                ct = ct + 1
                gradL2 = np.mean(grad_updated**2)
            return beta0
    if testtype == "likelihoodRatio":
        #========================fit the H0 model========================
        # A fits the initial model
        modelH0 = sm.GLM(endog = Y, exog = XAintercept, family = fitmodel)
        modelH0_results = modelH0.fit()

    # list of p-values
    pVList = []
    # list of rejection results
    rejList = []

    # initialize the predictor data for A
    XAintercept_add = XAintercept
    for i in range(K):
        # obtain the random vector u
        u = np.random.normal(loc = 0, scale = 1, size = (XB.shape[1], 1))
        uXB_tmp = np.dot(XB, u)
        # add noise
        uXB = uXB_tmp + np.random.laplace(loc=0, scale=Lapscale, size=uXB_tmp.shape)
  
        #XAintercept_add_tmp = np.concatenate((XAintercept_add, uXB.reshape(uXB.shape[0],-1)), axis = 1)
        XAintercept_add = np.concatenate((XAintercept_add, uXB.reshape(uXB.shape[0],-1)), axis = 1)
        # #=============delete linearly dependent columns=============
        # q,r = np.linalg.qr(XAintercept_add_tmp.T)

        # XAintercept_add = XAintercept_add_tmp[:, np.abs(np.diag(r))>=1e-10]
        #========================fit the H1 model========================
        # A fits the H1 model after receiving the data
        if Model == "logcosh":
            betaFitted = fitLogCosh(Y, XAintercept_add)
        else:
            modelH1 = sm.GLM(endog = Y, exog = XAintercept_add, family = fitmodel)
            modelH1_results = modelH1.fit()
            betaFitted = modelH1_results.params
        # test statistic
        if testtype == "likelihoodRatio":
            stat = 2 * (modelH1_results.llf - modelH0_results.llf)
        elif testtype == "Wald":
            # linear predictor
            lp = np.dot(XAintercept_add, betaFitted)
            
            # prediction result
            if Model == "logcosh":
                prd =  np.dot(XAintercept_add, betaFitted)
            else:
                prd = modelH1_results.predict(XAintercept_add)
            # number of predictors from A
            pA = XAintercept.shape[1]
            # fitted coefficients
            # prediction result
            betaU =betaFitted[pA: ]

            
            stat = statCal(Y, XAintercept_add, pA, prd, betaU, lp)
        df = (XAintercept_add.shape[1] - XAintercept.shape[1])
# #########
#         print(df)
#         df = i + 1
#         # print(modelH1_results.llf)
#         # print(modelH0_results.llf)
#         print(i+1)
# #########
        # p value
        pV = 1 - scipy.stats.chi2.cdf(stat, df)
        pVList.append(pV)
        # rejection
        rej = pV < 0.05
        rejList.append(rej)
        
    print(rejList)
    print(pVList)
    # [True, True, True]


In [3]:

np.random.seed(20)
for Lapscale in [0, 0.1, 0.5]:
    TestFun('Wald', Lapscale, 3, Y, XAintercept, XB)

# [True, True, True]
# [9.159139002790084e-10, 2.1220133428201393e-09, 4.6413983767479294e-12]
# [True, True, True]
# [0.0015097081663844047, 0.0003195817679734203, 1.1102230246251565e-16]
# [True, True, True]
# [0.0005485868072927502, 0.00630474822422511, 0.01961008254493568]
# >>> 0.05/3
# 0.016666666666666666

[True, True, True]
[9.159139002790084e-10, 2.1220133428201393e-09, 4.6413983767479294e-12]
[True, True, True]
[0.0015097081663844047, 0.0003195817679734203, 1.1102230246251565e-16]
[True, True, True]
[0.0005485868072927502, 0.00630474822422511, 0.01961008254493568]
